In [2]:
import requests

In [3]:
def json_url(_id):
    return f"https://pathbuilder2e.com/json.php?id={_id}"

def get_json(_id):
    rsp = requests.get(json_url(_id), headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:125.0) Gecko/20100101 Firefox/125.0"})
    rsp.raise_for_status()
    return rsp.json()

In [4]:
build = get_json(260556)["build"]

In [28]:
PACKS_DIR = "../foundry-pf2e/packs/"
PACKS_DIR_OLD = "../foundry-pf2e-old/packs/"

In [6]:
import os

In [7]:
import json

In [8]:
import logging

LOG = logging.getLogger(__name__)

In [29]:
packs = {}

import glob

files = glob.glob(os.path.join(PACKS_DIR_OLD, "**", "*.json"), recursive=True) + glob.glob(os.path.join(PACKS_DIR, "**", "*.json"), recursive=True)

for file in files:
    if not file.endswith(".json"):
        continue

    with open(file) as fd:
        obj = json.load(fd)

        if "name" not in obj:
            continue

        if "type" not in obj:
            continue

        packs.setdefault(obj["type"], {})[obj["name"]] = obj
        packs[obj["type"]][os.path.basename(file).split(".")[0]] = obj

In [30]:
basics_fields = ["name", "class", "dualClass", "level", "ancestry", "heritage", "background", "alignment", "gender", "age", "deity", "sizeName", "keyability", "languages", "resistances"]
basics = {k: build[k] for k in basics_fields}
basics["ac"] = build["acTotal"]["acTotal"]
basics["speed"] = build["attributes"]["speed"]

In [40]:
all_fields = {}

def traverse_dict(d, prefix=""):
    result = []
    for k, v in d.items():
        if isinstance(v, dict):
            result.extend(traverse_dict(v, prefix=f"{prefix}{k}."))
        else:
            result.append(f"{prefix}{k}")
    return result

for cat, items in packs.items():
    fields = set()
    for item in items.values():
        for field in traverse_dict(item):
            fields.add(field)
    all_fields[cat] = fields

In [43]:
build

{'name': 'Yorick',
 'class': 'Witch',
 'dualClass': None,
 'level': 11,
 'ancestry': 'Halfling',
 'heritage': 'Gutsy Halfling',
 'background': 'Cook',
 'alignment': 'Neutral Good',
 'gender': 'Male',
 'age': '25',
 'deity': 'Not set',
 'size': 1,
 'sizeName': 'Small',
 'keyability': 'int',
 'languages': ['Common', 'Elven', 'Halfling'],
 'rituals': [],
 'resistances': ['mental 5'],
 'attributes': {'ancestryhp': 6,
  'classhp': 6,
  'bonushp': 0,
  'bonushpPerLevel': 0,
  'speed': 25,
  'speedBonus': 0},
 'abilities': {'str': 10,
  'dex': 18,
  'con': 16,
  'int': 20,
  'wis': 18,
  'cha': 10,
  'breakdown': {'ancestryFree': ['Int'],
   'ancestryBoosts': ['Dex', 'Wis'],
   'ancestryFlaws': ['Str'],
   'backgroundBoosts': ['Con', 'Int'],
   'classBoosts': ['Int'],
   'mapLevelledBoosts': {'1': ['Dex', 'Int', 'Str', 'Wis'],
    '5': ['Con', 'Wis', 'Int', 'Dex'],
    '10': ['Dex', 'Wis', 'Con', 'Int']}}},
 'proficiencies': {'classDC': 4,
  'perception': 4,
  'fortitude': 4,
  'reflex': 4,
 

In [46]:
[(k, v) for k, pack in packs.items() for v in pack if "nudge" in v.lower()]

[('effect', 'Spell effect: Nudge the Odds'),
 ('effect', 'spell-effect-nudge-the-odds'),
 ('effect', 'Spell Effect: Nudge the Odds'),
 ('spell', 'Nudge the Odds'),
 ('spell', 'nudge-the-odds'),
 ('spell', 'Nudge Fate'),
 ('spell', 'nudge-fate')]

In [51]:
%pdb

Automatic pdb calling has been turned ON


In [58]:
cards = []

def find_in_pack(name, subsets):
    for subset in subsets:
        if name in packs[subset]:
            return packs[subset][name]
    msg = f"Couldn't find {name} in {subsets}"
    LOG.warning(msg)
    return {"name": name, "type": "error", "message": msg}

for feat in build["feats"]:
    name = feat[0]
    cards.append(find_in_pack(feat[0], ["feat", "heritage"]))

for caster in build.get("spellCasters", []):
    for spells in caster["spells"]:
        level = spells["spellLevel"]
        for name in spells["list"]:
            cards.append({
                **find_in_pack(name, ["spell"]),
                **{
                    "tradition": caster["magicTradition"],
                    "ability": caster["ability"],
                    "class": caster["name"],
                }
            })

for tradition_name, tradition in build.get("focus", {}).items():
    for ability_name, ability in tradition.items():
        for name in ability.get("focusCantrips", []):
            cards.append(dict(
                find_in_pack(name, ["spell"]),
                    tradition=tradition_name,
                    ability=ability_name,
                    spell_type="Focus Cantrip",
                )
            )
        for name in ability.get("focusSpells", []):
            cards.append(dict(
                find_in_pack(name, ["spell"]),
                    tradition=tradition_name,
                    ability=ability_name,
                    spell_type="Focus Spell",
                )
            )

Couldn't find Incredible Familiar in ['feat', 'heritage']
Couldn't find Major Lesson I in ['feat', 'heritage']
Couldn't find Shield (Amped) in ['spell']


In [59]:
cards

[{'_id': '1Eceqc6zbMj2x0q9',
  'img': 'icons/sundries/books/book-red-exclamation.webp',
  'name': 'Seasoned',
  'system': {'actionType': {'value': 'passive'},
   'actions': {'value': None},
   'category': 'skill',
   'description': {'value': "<p>You've mastered the preparation of many types of food and drink. You gain a +1 circumstance bonus to checks to @UUID[Compendium.pf2e.actionspf2e.Item.Craft] food and drink, including elixirs if you have @UUID[Compendium.pf2e.feats-srd.Item.Alchemical Crafting] and potions if you have @UUID[Compendium.pf2e.feats-srd.Item.Magical Crafting]. If you are a master in one of the prerequisite skills, this bonus increases to +2.</p>"},
   'level': {'value': 1},
   'prerequisites': {'value': [{'value': 'trained in Alcohol Lore, Cooking Lore, or Crafting'}]},
   'publication': {'license': 'ORC',
    'remaster': True,
    'title': 'Pathfinder Player Core'},
   'rules': [{'key': 'FlatModifier',
     'predicate': ['action:craft',
      {'or': ['item:tag:food